# CME 241: MIDTERM
#### Emile Clastres - clastres@stanford.edu

## Problem 1: Value Iteration Optimization

a)
The question is unclear abot whether our formulation should be general or relative to the provided maze. We are going to solve the provided maze, but let us state that it can be generalized by considering that the goal is located at position $(x_*, y_*)$ and that the grid size is from $0$ to $n$ instead of $0$ to $7$. In what follows, every variable $x, y$ is an integer.

The state-space is $\mathcal{S} = \{(x,y) | 0 \leq x,y \leq 7\}$, with $\mathcal{T} = \{(7,7)\}$
The unrestricted action space is $\mathcal{A} = \{u, d, l, r\}$ denoting UP, DOWN, LEFT and RIGHT. We will precise them further below.

Let us introduce two functions that are going to facilitate notations:

the function $space(s)$ is true if and only if the position $(x,y)$ is a space (or the goal), for $s \in \mathcal{S}$.
the function $move((x,y),a)$ returns $(x-1, y)$ if $a = u$, $(x+1, y)$ if $a = d$, $(x, y+1)$ if $a = r$, $(x, y-1)$ if $a = l$

We can now precise the restricted action space, filtering for illegal moves.

$\forall s\in \mathcal{S}, \mathcal{A}(s) = \{a | ~a \in \mathcal{A}:~ space(move(s,a)) ~\&~ move(s,a) \in \mathcal{S} \}$

With these notations, $\forall s \in \mathcal{S},~ \forall a \in \mathcal{A}(s), ~ \mathcal{P}(s,a,move(s,a)) = 1$

Note that this specification is not yet rigorous, as $\mathcal{P}(s, \cdot, \cdot)$ is not defined for $s$ that are blocks. We think that the above specification is however the clearest. 

The rigorous specification is:

$\mathcal{S} = \{(x,y) | 0 \leq x,y \leq 7 ~ \& ~ space(x,y)\}$

$\forall s\in \mathcal{S}, \mathcal{A}(s) = \{a | ~a \in \mathcal{A}: move(s,a) \in \mathcal{S} \}$
$\forall s \in \mathcal{S},~ \forall a \in \mathcal{A}(s), ~ \mathcal{P}(s,a,move(s,a)) = 1$

This specification is complete, and for exhaustivity we must state here that this definition assumes that it is not possible for a position $(x,y)$ to be surrounded by blocks unless it is the only position in S.


### Reward version 1:

$\gamma = 1$, $\forall s \in \mathcal{S},~ \forall a \in \mathcal{A}(s), ~ \mathcal{R}_T(s,a) = 1 ~~\text{if} ~~ move(s,a) \in \mathcal{T}, ~\text{else} ~ \mathcal{R}_T(s,a) = 0$

### Reward version 2:

$\gamma < 1$, $\forall s \in \mathcal{S},~ \forall a \in \mathcal{A}(s), ~ \mathcal{R}(s,a) = -1 $

b)

In [1]:
from typing import Tuple, Mapping, Sequence, Optional
from rl.markov_decision_process import FiniteMarkovDecisionProcess, FinitePolicy, StateActionMapping
from rl.distribution import Constant


SPACE = 'SPACE'
BLOCK = 'BLOCK'
GOAL = 'GOAL'

maze_grid = {(0, 0): SPACE, (0, 1): BLOCK, (0, 2): SPACE, (0, 3): SPACE, (0, 4): SPACE, 
             (0, 5): SPACE, (0, 6): SPACE, (0, 7): SPACE, (1, 0): SPACE, (1, 1): BLOCK,
             (1, 2): BLOCK, (1, 3): SPACE, (1, 4): BLOCK, (1, 5): BLOCK, (1, 6): BLOCK, 
             (1, 7): BLOCK, (2, 0): SPACE, (2, 1): BLOCK, (2, 2): SPACE, (2, 3): SPACE, 
             (2, 4): SPACE, (2, 5): SPACE, (2, 6): BLOCK, (2, 7): SPACE, (3, 0): SPACE, 
             (3, 1): SPACE, (3, 2): SPACE, (3, 3): BLOCK, (3, 4): BLOCK, (3, 5): SPACE, 
             (3, 6): BLOCK, (3, 7): SPACE, (4, 0): SPACE, (4, 1): BLOCK, (4, 2): SPACE, 
             (4, 3): BLOCK, (4, 4): SPACE, (4, 5): SPACE, (4, 6): SPACE, (4, 7): SPACE, 
             (5, 0): BLOCK, (5, 1): BLOCK, (5, 2): SPACE, (5, 3): BLOCK, (5, 4): SPACE, 
             (5, 5): BLOCK, (5, 6): SPACE, (5, 7): BLOCK, (6, 0): SPACE, (6, 1): BLOCK, 
             (6, 2): BLOCK, (6, 3): BLOCK, (6, 4): SPACE, (6, 5): BLOCK, (6, 6): SPACE, 
             (6, 7): SPACE, (7, 0): SPACE, (7, 1): SPACE, (7, 2): SPACE, (7, 3): SPACE, 
             (7, 4): SPACE, (7, 5): BLOCK, (7, 6): BLOCK, (7, 7): GOAL}

UP = 'UP'
DOWN = 'DOWN'
LEFT = 'LEFT'
RIGHT = 'RIGHT'

ACTIONS = [UP, DOWN, LEFT, RIGHT]

Action = str
Position = Tuple[int, int]

class MazeMDP(FiniteMarkovDecisionProcess):
    gamma : float
    
    def __init__(self, maze_grid : Mapping[Position, Action] = maze_grid,
                         gamma : float = 1.):        
        self.gamma = gamma
        n : int = max(x for x,_ in maze_grid)
        m : int = max(y for _,y in maze_grid)
        
        def reward(s) -> float:
            """reward gained from arriving at state s"""
            if gamma >= 1.:
                return -1.
            else:
                return float(maze_grid[s] is GOAL)
            
        def available_actions(s) -> Sequence[Action]:
            return [a for a in ACTIONS if (move(s,a) in maze_grid) and \
                                (maze_grid[move(s,a)] in [SPACE, GOAL])]
    
        def move(s : Position, a : Action) -> Position:
            """Deterministic movement from state s with direction a"""
            x,y = s
            assert a in ACTIONS
            if a == UP:
                return (x-1, y)
            elif a == DOWN:
                return (x+1, y)
            elif a == LEFT:
                return (x, y-1)
            else:
                return (x, y+1)
        
        sa_mapping : StateActionMapping[Position, Action, Position] = {}
        for x in range(n + 1):
            for y in range(m + 1):
                s = (x,y)
                if s in maze_grid and maze_grid[s] is SPACE:
                    sa_mapping[s] = {}
                    for a in available_actions(s):
                        s_ = move(s, a)
                        sa_mapping[s][a] = Constant((s_, reward(s_)))
                    if len(available_actions(s)) == 0:
                        sa_mapping[s][a] = None
                elif s is GOAL:
                    sa_mapping[s] = None

        super().__init__(sa_mapping)
        
    
        


In [2]:
from rl.dynamic_programming import value_iteration_result

We modified the source code for value_iteration_result in order to take an optionnal argument print_steps which, if True, prints the number of iterations.
The source code is available on my repo as well as in my Assignment 4.

It is copied below, but needs to be present in rl.dynamic_programming and rl.iteration to be run



In [ ]:
def converged_counted(values: Iterator[X],
                    done: Callable[[X, X], bool]) -> X:
    '''Return the final value of the given iterator and prints 
    the number of iterations when its values
    converge according to the done function.

    Raises an error if the iterator is empty.

    Will loop forever if the input iterator doesn't end *or* converge.
    '''
    result = None
    count:int = 0
    for val in converge(values, done):
        count+=1
        result = val

    if result is None:
        raise ValueError("converged called on an empty iterator")
    else:
        print(f"Converged in {n} iterations.")
    return result


def policy_iteration_result(
    mdp: FiniteMarkovDecisionProcess[S, A],
    gamma: float,
    print_steps: bool = False
) -> Tuple[V[S], FinitePolicy[S, A]]:
	if not print_steps:
    	return converged(policy_iteration(mdp, gamma), done=almost_equal_vf_pis)
    else:
    	return converged_counted(policy_iteration(mdp, gamma), done=almost_equal_vf_pis)


def value_iteration_result(
    mdp: FiniteMarkovDecisionProcess[S, A],
    gamma: float,
    print_steps: bool = False
) -> Tuple[V[S], FinitePolicy[S, A]]:
	if not print_steps:
	    opt_vf: V[S] = converged(
	        value_iteration(mdp, gamma),
	        done=almost_equal_vfs
	    )
	else:
		opt_vf: V[S] = converged_counted(
	        value_iteration(mdp, gamma),
	        done=almost_equal_vfs
	    )
    opt_policy: FinitePolicy[S, A] = greedy_policy_from_vf(
        mdp,
        opt_vf,
        gamma
    )

    return opt_vf, opt_policy

### Let's take a look at the convergence speed depending on gamma:

In [3]:
maze_v1 = MazeMDP(maze_grid = maze_grid, gamma  = 1.)
opt_vf_v1, opt_policy_v1 = value_iteration_result(mdp = maze_v1, gamma = maze_v1.gamma, print_steps = True)

Converged in 17 iterations.


In [4]:
print(opt_vf_v1.__repr__()[:60])

{(0, 0): -16.0, (0, 2): -12.0, (0, 3): -11.0, (0, 4): -12.0,


In [5]:
maze_v2 = MazeMDP(maze_grid = maze_grid, gamma  =  0.9)
opt_vf_v2, opt_policy_v2 = value_iteration_result(mdp = maze_v2, gamma = maze_v2.gamma, print_steps = True)

Converged in 17 iterations.


In [6]:
print(opt_vf_v2.__repr__()[:60])

{(0, 0): 0.2058911320946491, (0, 2): 0.31381059609000017, (0


##### Remark : setting $\gamma$ to a low value (e.g 0.1) reduces the number of iterations to 7, but the Value function becomes so close to zero that it is likely float approximations errors are occuring.

The two methods converge in 17 iterations, and they produce (as they should) the same deterministic policies.
The below cell checks the last claim.

In [7]:
def same_policy(pi_1 : FinitePolicy, pi_2 : FinitePolicy) -> bool:
    return all([pi_1.states() == pi_2.states()] +\
               [(pi_1.act(s) is None and  pi_2.act(s) is None) or\
                (pi_1.act(s).table == pi_2.act(s).table) for s in pi_1.states()])
same_policy(opt_policy_v2, opt_policy_v1)

True

## Problem 2: MRP Value Function Approximation

We know that the Value function (denoted V) can be characterized as the only fixed point of the Bellman operator (Banach fixed-point Theorem).
Let $\Phi$ be a feature matrix, the following equivalences hold:

We can approximate exactly $V$  
$\Leftrightarrow \exists \beta \in R^n, ~ V = \Phi \beta $  
$\Leftrightarrow \exists \beta \in R^n, ~ \Phi \beta = \mathcal{R} + \gamma \mathcal{P}\Phi \beta $  
$\Leftrightarrow \exists \beta \in R^n, ~ \Phi \beta =  ( \mathbf{I} - \gamma \mathcal{P}\Phi )^{-1} \mathcal{R} $  
$\Leftrightarrow ( \mathbf{I} - \gamma \mathcal{P}\Phi )^{-1} \mathcal{R} \in \text{Range}(\Phi)$



Therefore, a minimal condition for V to be exactly linearly approximated with the design matrix $\Phi$ is that $( \mathbf{I} - \gamma \mathcal{P}\Phi )^{-1} \mathcal{R}$ should be in the Range of $\Phi$


Remark : let $\gamma = 0$, this condition reduces to the existence of a perfect linear relationship between $\Phi$ and $\mathcal{R}$ : $\mathcal{R} = \Phi \beta$

## Problem 3: Career Optimization

In what follows, $l$ and $s$ always denote non-negative integers. $\alpha$ is a fixed postive number.

* The state space is the wage-space : $\mathcal{S} = \{1, ~...~,~ W\}$, with no terminating state.
* The action space is the schedule-space : $\mathcal{A} = \{(l,s), ~ 0 \leq l + s \leq H\}$
* $\forall w \in \mathcal{S},~ \forall (l,s) \in \mathcal{A}$:  

$ \text{if} ~ w < W, ~ \mathcal{P}(w,(l,s), w) = (1 - \frac{\beta s}{H})e^{-\alpha l}$  

$ \text{if} ~ w + 1 < W, ~ \mathcal{P}(w,(l,s), w + 1) = (\frac{\beta s}{H} + \alpha l)e^{-\alpha l} $  

$  \forall x \in \mathbb{Z}_{\geq 2},~\text{if} ~ w + x < W,  ~ \mathcal{P}(w,(l,s), w + x) = \frac{(\alpha l)^xe^{-\alpha l}}{x!}$  

$\mathcal{P}(w,(l,s), W) = 1 - \sum_{0 \leq x \leq W-w -1} \mathcal{P}(w,(l,s), w+x)$  


* $\forall w \in \mathcal{S},~ \forall (l,s) \in \mathcal{A} ,~ \mathcal{R}_T(w, (l,s))= w (H - l - s)$
*  $0 \leq \gamma < 1$, for example $\gamma = 0.5^{1/30} \approx 0.977$ would correspond to giving a 50% discount after exactly one month.

In [8]:
from rl.distribution import Categorical
from numpy.random import randint
from math import exp
Wage = int
DayPlan = Tuple[int, int]

#remark : we will use b (book) in place of l (learn) to avoid confusion with the number 1.

class CareerMDP(FiniteMarkovDecisionProcess):
    def __init__(self, alpha : float = 0.08, beta : float = 0.82 ,W : int = 30, H:int = 10):
        # pre-computation of factorials
        factorial = [1]
        for i in range(1,W+1):
            factorial.append(factorial[-1]*i)
            
        def proba(a : DayPlan, x : int) -> float:
            b,s = a
            """returns the probability of going from w to w+x with w+x < W"""
            res = (alpha * b)**x * exp(-alpha*b) / factorial[x]
            if x == 0:
                res *= (1-(beta * s / H))
            elif x== 1:
                res += (beta * s / H)*exp(-alpha*b)
            return res
        
        sa_mapping : StateActionMapping[Wage, DayPlan, Wage] = {}
        for w in range(1,W+1):
            sa_mapping[w] = {}
            for b in range(H+1):
                for s in range(H - b):
                    #state-reward probabilities
                    proba_dict = {(w+x, (H-b-s)*w): proba((b,s),x) for x in range(W-w)\
                                  if proba((b,s),x) > 0}
                    total = sum(proba_dict.values())
                    if total < 1. :
                        proba_dict[(W, (H-b-s)*w)] =  (1. - total)
                    sa_mapping[w][(b,s)] = Categorical(proba_dict)
        super().__init__(sa_mapping)

In [9]:
career_mdp = CareerMDP(W = 30, H = 10, alpha = 0.08, beta = 0.82)

In [10]:
vf, pi = value_iteration_result(mdp = career_mdp, gamma = 0.95, print_steps = True)

Converged in 337 iterations.


The optimal policy is :

In [11]:
pi

For State 1:
  Do Action (0, 9) with Probability 1.000
For State 2:
  Do Action (0, 9) with Probability 1.000
For State 3:
  Do Action (0, 9) with Probability 1.000
For State 4:
  Do Action (0, 9) with Probability 1.000
For State 5:
  Do Action (0, 9) with Probability 1.000
For State 6:
  Do Action (0, 9) with Probability 1.000
For State 7:
  Do Action (0, 9) with Probability 1.000
For State 8:
  Do Action (0, 9) with Probability 1.000
For State 9:
  Do Action (0, 9) with Probability 1.000
For State 10:
  Do Action (0, 9) with Probability 1.000
For State 11:
  Do Action (0, 9) with Probability 1.000
For State 12:
  Do Action (0, 9) with Probability 1.000
For State 13:
  Do Action (0, 9) with Probability 1.000
For State 14:
  Do Action (0, 9) with Probability 1.000
For State 15:
  Do Action (0, 9) with Probability 1.000
For State 16:
  Do Action (0, 0) with Probability 1.000
For State 17:
  Do Action (0, 0) with Probability 1.000
For State 18:
  Do Action (0, 0) with Probability 1.000
F

The optimal policy is to spend 100% of one's time searching for a new job if he isn't paid at least 16, and to work fulltime otherwise. The rationale behind this is that since alpha is very low, the discounted expectation  of raise from searching is larger. As long as the expected raise from a search time is greater than one's salary, he prefers to study fulltime. As soon as it's not the case there is no incentive to study anymore.

It is pretty sensible that either searching or learning is strictly better than the other (it depends on alpha, cranking it up will replace the role of s with l).
To get the right intuition on the actual policy let's study the two extreme cases.
* Hungry worker : is only interested in today's pay ($\gamma = 0$) : will always work (l = s = 0)
* Disinterested worker : is only interested in maximizing daily income, but doesn't care to wait as long as necessary : will always spend all his time getting to the next salary.

In our case, it is a tradeoff between the two extremes, where the agent is willing to increase its wage, up until the time spent increasing that is not worth the salary loss with regards to his patience.

This is a pretty short sighted strategy, as obviously in the (very) long run it is worth it to spend the first 30 days getting to W. Let's test that guess by setting $\gamma = 0.99$:

In [12]:
vf, pi = value_iteration_result(mdp = career_mdp, gamma = 0.99);pi

For State 1:
  Do Action (0, 9) with Probability 1.000
For State 2:
  Do Action (0, 9) with Probability 1.000
For State 3:
  Do Action (0, 9) with Probability 1.000
For State 4:
  Do Action (0, 9) with Probability 1.000
For State 5:
  Do Action (0, 9) with Probability 1.000
For State 6:
  Do Action (0, 9) with Probability 1.000
For State 7:
  Do Action (0, 9) with Probability 1.000
For State 8:
  Do Action (0, 9) with Probability 1.000
For State 9:
  Do Action (0, 9) with Probability 1.000
For State 10:
  Do Action (0, 9) with Probability 1.000
For State 11:
  Do Action (0, 9) with Probability 1.000
For State 12:
  Do Action (0, 9) with Probability 1.000
For State 13:
  Do Action (0, 9) with Probability 1.000
For State 14:
  Do Action (0, 9) with Probability 1.000
For State 15:
  Do Action (0, 9) with Probability 1.000
For State 16:
  Do Action (0, 9) with Probability 1.000
For State 17:
  Do Action (0, 9) with Probability 1.000
For State 18:
  Do Action (0, 9) with Probability 1.000
F

As expected, the agent now is willing to take the effort to rise to the top.
Let's verify that increasing alpha will change the roles of l and s:


In [13]:
career_mdp = CareerMDP(W = 30, H = 10, alpha = 0.16, beta = 0.82)
vf, pi = value_iteration_result(mdp = career_mdp, gamma = 0.95, print_steps = True)

Converged in 337 iterations.


In [14]:
pi

For State 1:
  Do Action (9, 0) with Probability 1.000
For State 2:
  Do Action (9, 0) with Probability 1.000
For State 3:
  Do Action (9, 0) with Probability 1.000
For State 4:
  Do Action (9, 0) with Probability 1.000
For State 5:
  Do Action (9, 0) with Probability 1.000
For State 6:
  Do Action (9, 0) with Probability 1.000
For State 7:
  Do Action (9, 0) with Probability 1.000
For State 8:
  Do Action (9, 0) with Probability 1.000
For State 9:
  Do Action (9, 0) with Probability 1.000
For State 10:
  Do Action (9, 0) with Probability 1.000
For State 11:
  Do Action (9, 0) with Probability 1.000
For State 12:
  Do Action (9, 0) with Probability 1.000
For State 13:
  Do Action (9, 0) with Probability 1.000
For State 14:
  Do Action (9, 0) with Probability 1.000
For State 15:
  Do Action (9, 0) with Probability 1.000
For State 16:
  Do Action (9, 0) with Probability 1.000
For State 17:
  Do Action (9, 0) with Probability 1.000
For State 18:
  Do Action (9, 0) with Probability 1.000
F

As expected the agent now only Learns instead of searching, with a slight difference : when approaching W, he spends less time learning and some time working. This is because the learning action is stochastic with a tail collapse at W, and a mode at 0, which makes the expected reward from learning non-linear with the time spent and the distance to W.